In [2]:
## v1 실패

import random
from itertools import combinations

# 선수 목록 생성
men = [f"남{i+1}" for i in range(22)]
women = [f"여{i+1}" for i in range(10)]

# 설정
total_matches_per_player = 4
court_capacity = 4
num_courts = 4
total_rounds = 8

# 각 선수의 경기 횟수 및 파트너와 만남 횟수 추적
players_matches = {player: 0 for player in men + women}
partner_counts = {player: {} for player in men + women}

# 경기 타입에 따른 선수 그룹화
def add_match(match_type, players):
    for player in players:
        players_matches[player] += 1
        for partner in players:
            if partner != player:
                if partner not in partner_counts[player]:
                    partner_counts[player][partner] = 0
                partner_counts[player][partner] += 1
    return (match_type, players)

# 혼합복식, 남자복식, 여자복식 매치 생성
def create_schedule(men, women):
    schedule = []

    # 조건을 만족하는 매치 리스트 준비
    mixed_double_matches = [
        add_match("혼합복식", list(combo))
        for combo in combinations(men, 2) for _ in combinations(women, 2)
    ]
    men_double_matches = [
        add_match("남자복식", list(combo)) for combo in combinations(men, 4)
    ]
    women_double_matches = [
        add_match("여자복식", list(combo)) for combo in combinations(women, 4)
    ]

    # 각 타임별로 매치를 무작위로 선정
    for _ in range(total_rounds):
        round_matches = []
        while len(round_matches) < num_courts:
            match_type = random.choice(["혼합복식", "남자복식", "여자복식"])
            if match_type == "혼합복식" and mixed_double_matches:
                match = mixed_double_matches.pop()
            elif match_type == "남자복식" and men_double_matches:
                match = men_double_matches.pop()
            elif match_type == "여자복식" and women_double_matches:
                match = women_double_matches.pop()
            else:
                continue  # 선택된 타입의 매치가 없는 경우 반복

            # 같은 파트너 3회 이상 제한 체크
            if all(partner_counts[p1].get(p2, 0) < 3 for p1 in match[1] for p2 in match[1] if p1 != p2):
                round_matches.append(match)

        schedule.append(round_matches)

    return schedule

schedule = create_schedule(men, women)

# 출력
for round_num, round_matches in enumerate(schedule, start=1):
    print(f"Round {round_num}")
    for court_num, (match_type, players) in enumerate(round_matches, start=1):
        print(f"  Court {court_num}: {match_type} - {players}")


KeyboardInterrupt: 

In [11]:
import random
from itertools import combinations

# 선수 목록 생성
men = [f"남{i+1}" for i in range(22)]
women = [f"여{i+1}" for i in range(10)]

# 설정
total_matches_per_player = 4
num_courts = 4
total_rounds = 8

# 각 선수의 경기 횟수 및 파트너 만남 횟수 추적
players_matches = {player: 0 for player in men + women}
partner_counts = {player: {} for player in men + women}
match_type_count = {player: {'혼성복식': 0, '남자복식': 0, '여자복식': 0} for player in men + women}

# 파트너 만남 횟수를 기록하는 함수
def record_match(players, match_type):
    for player in players:
        players_matches[player] += 1
        match_type_count[player][match_type] += 1
        for partner in players:
            if partner != player:
                if partner not in partner_counts[player]:
                    partner_counts[player][partner] = 0
                partner_counts[player][partner] += 1

# 조건 충족 확인 함수
def is_valid_match(players, match_type):
    for player in players:
        # 최대 경기 횟수 및 파트너와의 만남 횟수 확인
        if players_matches[player] >= total_matches_per_player:
            return False
        if any(partner_counts[p1].get(p2, 0) >= 3 for p1 in players for p2 in players if p1 != p2):
            return False
        
        # 여자는 혼성복식 2회, 여자복식 2회 조건 확인
        if player in women:
            if match_type == "혼성복식" and match_type_count[player]["혼성복식"] >= 2:
                return False
            if match_type == "여자복식" and match_type_count[player]["여자복식"] >= 2:
                return False
            
        # 남자는 혼성복식 2회와 남자복식 2회 또는 혼성복식 1회와 남자복식 3회 조건 확인
        if player in men:
            if match_type == "혼성복식" and match_type_count[player]["혼성복식"] >= 2:
                return False
            if match_type == "남자복식":
                if match_type_count[player]["남자복식"] == 3 and match_type_count[player]["혼성복식"] == 1:
                    return False
                if match_type_count[player]["남자복식"] == 2 and match_type_count[player]["혼성복식"] == 2:
                    return False
    return True

# 가능한 매치 조합을 미리 생성
mixed_double_matches = [
    list(combo) for combo in combinations(men, 2) for _ in combinations(women, 2)
]
men_double_matches = [list(combo) for combo in combinations(men, 4)]
women_double_matches = [list(combo) for combo in combinations(women, 4)]

# 경기 스케줄 생성
def create_schedule():
    schedule = []

    for _ in range(total_rounds):
        round_matches = []
        
        # 각 코트에 맞는 매치 타입을 미리 배정
        court_matches = random.choices(["혼성복식", "남자복식", "여자복식"], k=num_courts)

        for match_type in court_matches:
            # 각 매치 타입에 맞는 가능한 매치 리스트에서 조건을 충족하는 매치를 선택
            if match_type == "혼성복식":
                valid_matches = [match for match in mixed_double_matches if is_valid_match(match, "혼성복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    mixed_double_matches.remove(match)
            elif match_type == "남자복식":
                valid_matches = [match for match in men_double_matches if is_valid_match(match, "남자복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    men_double_matches.remove(match)
            elif match_type == "여자복식":
                valid_matches = [match for match in women_double_matches if is_valid_match(match, "여자복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    women_double_matches.remove(match)
            else:
                continue

            # 매치 기록 및 경기 배정
            record_match(match, match_type)
            round_matches.append((match_type, match))

        schedule.append(round_matches)

    return schedule

# 스케줄 생성 및 출력
schedule = create_schedule()

for round_num, round_matches in enumerate(schedule, start=1):
    print(f"Round {round_num}")
    for court_num, (match_type, players) in enumerate(round_matches, start=1):
        print(f"  Court {court_num}: {match_type} - {players}")

# 각 선수별 경기 횟수 및 경기 타입 확인
for player, count in players_matches.items():
    print(f"{player} has {count} matches (혼성복식: {match_type_count[player]['혼성복식']}, 남자복식: {match_type_count[player]['남자복식']}, 여자복식: {match_type_count[player]['여자복식']})")


Round 1
  Court 1: 여자복식 - ['여5', '여7', '여9', '여10']
  Court 2: 혼성복식 - ['남5', '남15']
  Court 3: 남자복식 - ['남5', '남6', '남10', '남21']
  Court 4: 혼성복식 - ['남10', '남14']
Round 2
  Court 1: 여자복식 - ['여2', '여4', '여6', '여9']
  Court 2: 남자복식 - ['남7', '남8', '남10', '남21']
  Court 3: 여자복식 - ['여2', '여4', '여8', '여10']
  Court 4: 남자복식 - ['남1', '남11', '남14', '남21']
Round 3
  Court 1: 여자복식 - ['여1', '여3', '여5', '여8']
  Court 2: 혼성복식 - ['남16', '남22']
  Court 3: 남자복식 - ['남1', '남4', '남18', '남22']
  Court 4: 남자복식 - ['남3', '남7', '남9', '남13']
Round 4
  Court 1: 남자복식 - ['남7', '남12', '남17', '남22']
  Court 2: 혼성복식 - ['남4', '남15']
  Court 3: 남자복식 - ['남5', '남6', '남8', '남10']
  Court 4: 혼성복식 - ['남8', '남17']
Round 5
  Court 1: 혼성복식 - ['남12', '남17']
  Court 2: 여자복식 - ['여1', '여3', '여6', '여7']
  Court 3: 남자복식 - ['남4', '남8', '남12', '남14']
  Court 4: 혼성복식 - ['남2', '남7']
Round 6
  Court 1: 혼성복식 - ['남1', '남19']
  Court 2: 여자복식 - ['남1', '남19']
  Court 3: 여자복식 - ['남1', '남19']
  Court 4: 여자복식 - ['남1', '남19']
Round 7
  Court 1: 혼성

In [14]:
import random
from itertools import combinations

# 선수 목록 생성
men = [f"남{i+1}" for i in range(22)]
women = [f"여{i+1}" for i in range(10)]

# 설정
total_matches_per_player = 4
num_courts = 4
total_rounds = 8

# 각 선수의 경기 횟수 및 파트너 만남 횟수 추적
players_matches = {player: 0 for player in men + women}
partner_counts = {player: {} for player in men + women}
match_type_count = {player: {'혼성복식': 0, '남자복식': 0, '여자복식': 0} for player in men + women}

# 파트너 만남 횟수를 기록하는 함수
def record_match(players, match_type):
    for player in players:
        players_matches[player] += 1
        match_type_count[player][match_type] += 1
        for partner in players:
            if partner != player:
                if partner not in partner_counts[player]:
                    partner_counts[player][partner] = 0
                partner_counts[player][partner] += 1

# 조건 충족 확인 함수
def is_valid_match(players, match_type):
    for player in players:
        # 최대 경기 횟수 및 파트너와의 만남 횟수 확인
        if players_matches[player] >= total_matches_per_player:
            return False
        if any(partner_counts[p1].get(p2, 0) >= 3 for p1 in players for p2 in players if p1 != p2):
            return False
        
        # 여자는 혼성복식 2회, 여자복식 2회 조건 확인
        if player in women:
            if match_type == "혼성복식" and match_type_count[player]["혼성복식"] >= 2:
                return False
            if match_type == "여자복식" and match_type_count[player]["여자복식"] >= 2:
                return False
            
        # 남자는 혼성복식 2회와 남자복식 2회 또는 혼성복식 1회와 남자복식 3회 조건 확인
        if player in men:
            if match_type == "혼성복식" and match_type_count[player]["혼성복식"] >= 2:
                return False
            if match_type == "남자복식":
                if match_type_count[player]["남자복식"] == 3 and match_type_count[player]["혼성복식"] == 1:
                    return False
                if match_type_count[player]["남자복식"] == 2 and match_type_count[player]["혼성복식"] == 2:
                    return False
    return True

# 가능한 매치 조합을 미리 생성
mixed_double_matches = [
    list(combo) for men_combo in combinations(men, 2) for women_combo in combinations(women, 2)
    for combo in [men_combo + women_combo]
]
men_double_matches = [list(combo) for combo in combinations(men, 4)]
women_double_matches = [list(combo) for combo in combinations(women, 4)]

# 경기 스케줄 생성
def create_schedule():
    schedule = []

    for _ in range(total_rounds):
        round_matches = []
        
        # 각 코트에 맞는 매치 타입을 미리 배정
        court_matches = random.choices(["혼성복식", "남자복식", "여자복식"], k=num_courts)

        for match_type in court_matches:
            # 각 매치 타입에 맞는 가능한 매치 리스트에서 조건을 충족하는 매치를 선택
            if match_type == "혼성복식":
                valid_matches = [match for match in mixed_double_matches if is_valid_match(match, "혼성복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    mixed_double_matches.remove(match)
            elif match_type == "남자복식":
                valid_matches = [match for match in men_double_matches if is_valid_match(match, "남자복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    men_double_matches.remove(match)
            elif match_type == "여자복식":
                valid_matches = [match for match in women_double_matches if is_valid_match(match, "여자복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    women_double_matches.remove(match)
            else:
                continue

            # 매치 기록 및 경기 배정
            record_match(match, match_type)
            round_matches.append((match_type, match))

        schedule.append(round_matches)

    return schedule

# 스케줄 생성 및 출력
schedule = create_schedule()

for round_num, round_matches in enumerate(schedule, start=1):
    print(f"Round {round_num}")
    for court_num, (match_type, players) in enumerate(round_matches, start=1):
        print(f"  Court {court_num}: {match_type} - {players}")

# 각 선수별 경기 횟수 및 경기 타입 확인
for player, count in players_matches.items():
    print(f"{player} has {count} matches (혼성복식: {match_type_count[player]['혼성복식']}, 남자복식: {match_type_count[player]['남자복식']}, 여자복식: {match_type_count[player]['여자복식']})")


Round 1
  Court 1: 남자복식 - ['남14', '남15', '남16', '남19']
  Court 2: 남자복식 - ['남8', '남12', '남14', '남20']
  Court 3: 여자복식 - ['여2', '여3', '여5', '여8']
  Court 4: 여자복식 - ['여1', '여4', '여5', '여9']
Round 2
  Court 1: 여자복식 - ['여3', '여6', '여9', '여10']
  Court 2: 남자복식 - ['남6', '남9', '남10', '남18']
  Court 3: 여자복식 - ['여1', '여2', '여7', '여8']
  Court 4: 남자복식 - ['남7', '남8', '남19', '남22']
Round 3
  Court 1: 여자복식 - ['여4', '여6', '여7', '여10']
  Court 2: 남자복식 - ['남4', '남8', '남12', '남21']
  Court 3: 여자복식 - ['남4', '남8', '남12', '남21']
  Court 4: 여자복식 - ['남4', '남8', '남12', '남21']
Round 4
  Court 1: 남자복식 - ['남7', '남11', '남14', '남19']
  Court 2: 여자복식 - ['남7', '남11', '남14', '남19']
  Court 3: 혼성복식 - ['남4', '남18', '여7', '여9']
  Court 4: 남자복식 - ['남2', '남15', '남20', '남22']
Round 5
  Court 1: 여자복식 - ['남2', '남15', '남20', '남22']
  Court 2: 여자복식 - ['남2', '남15', '남20', '남22']
  Court 3: 혼성복식 - ['남7', '남18', '여9', '여10']
  Court 4: 남자복식 - ['남3', '남6', '남17', '남18']
Round 6
  Court 1: 혼성복식 - ['남13', '남17', '여2', '여3']
  Court 

In [17]:
import random
from itertools import combinations

# 선수 목록 생성
men = [f"남{i+1}" for i in range(22)]
women = [f"여{i+1}" for i in range(10)]

# 설정
total_matches_per_player = 4
num_courts = 4
total_rounds = 8

# 각 선수의 경기 횟수 및 파트너 만남 횟수 추적
players_matches = {player: 0 for player in men + women}
partner_counts = {player: {} for player in men + women}
match_type_count = {player: {'혼성복식': 0, '남자복식': 0, '여자복식': 0} for player in men + women}

# 파트너 만남 횟수를 기록하는 함수
def record_match(players, match_type):
    for player in players:
        players_matches[player] += 1
        match_type_count[player][match_type] += 1
        for partner in players:
            if partner != player:
                if partner not in partner_counts[player]:
                    partner_counts[player][partner] = 0
                partner_counts[player][partner] += 1

# 조건 충족 확인 함수
def is_valid_match(players, match_type):
    for player in players:
        # 최대 경기 횟수 확인
        if players_matches[player] >= total_matches_per_player:
            return False
        # 최대 파트너 만남 횟수 확인
        if any(partner_counts[p1].get(p2, 0) >= 3 for p1 in players for p2 in players if p1 != p2):
            return False
        
        # 여자는 혼성복식 2회, 여자복식 2회 조건 확인
        if player in women:
            if match_type == "혼성복식" and match_type_count[player]["혼성복식"] >= 2:
                return False
            if match_type == "여자복식" and match_type_count[player]["여자복식"] >= 2:
                return False
            
        # 남자는 혼성복식 2회와 남자복식 2회 또는 혼성복식 1회와 남자복식 3회 조건 확인
        if player in men:
            if match_type == "혼성복식" and match_type_count[player]["혼성복식"] >= 2:
                return False
            if match_type == "남자복식":
                if match_type_count[player]["남자복식"] == 3 and match_type_count[player]["혼성복식"] == 1:
                    return False
                if match_type_count[player]["남자복식"] == 2 and match_type_count[player]["혼성복식"] == 2:
                    return False
    return True

# 가능한 매치 조합을 미리 생성
mixed_double_matches = [
    list(combo) for men_combo in combinations(men, 2) for women_combo in combinations(women, 2)
    for combo in [men_combo + women_combo]
]
men_double_matches = [list(combo) for combo in combinations(men, 4)]
women_double_matches = [list(combo) for combo in combinations(women, 4)]

# 경기 스케줄 생성
def create_schedule():
    schedule = []

    for _ in range(total_rounds):
        round_matches = []
        
        # 각 코트에 맞는 매치 타입을 미리 배정
        court_matches = random.choices(["혼성복식", "남자복식", "여자복식"], k=num_courts)

        for match_type in court_matches:
            # 각 매치 타입에 맞는 가능한 매치 리스트에서 조건을 충족하는 매치를 선택
            if match_type == "혼성복식":
                valid_matches = [match for match in mixed_double_matches if is_valid_match(match, "혼성복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    mixed_double_matches.remove(match)
            elif match_type == "남자복식":
                valid_matches = [match for match in men_double_matches if is_valid_match(match, "남자복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    men_double_matches.remove(match)
            elif match_type == "여자복식":
                valid_matches = [match for match in women_double_matches if is_valid_match(match, "여자복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    women_double_matches.remove(match)
            else:
                continue

            # 매치 기록 및 경기 배정
            record_match(match, match_type)
            round_matches.append((match_type, match))

        schedule.append(round_matches)

    # 각 선수의 경기 횟수가 정확히 4번인지 확인
    for player in players_matches:
        if players_matches[player] != total_matches_per_player:
            print(f"Error: {player} has {players_matches[player]} matches instead of {total_matches_per_player}")
            return None

    return schedule

# 스케줄 생성 및 출력
schedule = create_schedule()

if schedule:
    for round_num, round_matches in enumerate(schedule, start=1):
        print(f"Round {round_num}")
        for court_num, (match_type, players) in enumerate(round_matches, start=1):
            print(f"  Court {court_num}: {match_type} - {players}")

    # 각 선수별 경기 횟수 및 경기 타입 확인
    for player, count in players_matches.items():
        print(f"{player} has {count} matches (혼성복식: {match_type_count[player]['혼성복식']}, 남자복식: {match_type_count[player]['남자복식']}, 여자복식: {match_type_count[player]['여자복식']})")
else:
    print("스케줄 생성에 실패했습니다. 조건을 충족하는 구성을 찾을 수 없습니다.")


Error: 남2 has 6 matches instead of 4
스케줄 생성에 실패했습니다. 조건을 충족하는 구성을 찾을 수 없습니다.


In [25]:
import random
from itertools import combinations

# 선수 목록 생성
men = [f"남{i+1}" for i in range(22)]
women = [f"여{i+1}" for i in range(10)]

# 설정
total_matches_per_player = 4
num_courts = 4
total_rounds = 8

# 각 선수의 경기 횟수 및 파트너 만남 횟수 추적
players_matches = {player: 0 for player in men + women}
partner_counts = {player: {} for player in men + women}
match_type_count = {player: {'혼성복식': 0, '남자복식': 0, '여자복식': 0} for player in men + women}

# 파트너 만남 횟수를 기록하는 함수
def record_match(players, match_type):
    for player in players:
        players_matches[player] += 1
        match_type_count[player][match_type] += 1
        for partner in players:
            if partner != player:
                if partner not in partner_counts[player]:
                    partner_counts[player][partner] = 0
                partner_counts[player][partner] += 1

# 조건 충족 확인 함수
def is_valid_match(players, match_type, allow_flexible_pairs=False):
    for player in players:
        # 최대 경기 횟수 확인
        if players_matches[player] >= total_matches_per_player:
            return False
        
        # 파트너와의 만남 횟수 제한 확인, 필요 시 완화
        if not allow_flexible_pairs and any(partner_counts[p1].get(p2, 0) >= 3 for p1 in players for p2 in players if p1 != p2):
            return False
        
        # 여자는 혼성복식 2회, 여자복식 2회 조건 확인
        if player in women:
            if match_type == "혼성복식" and match_type_count[player]["혼성복식"] >= 2:
                return False
            if match_type == "여자복식" and match_type_count[player]["여자복식"] >= 2:
                return False
            
        # 남자는 혼성복식 2회와 남자복식 2회 또는 혼성복식 1회와 남자복식 3회 조건 확인
        if player in men:
            if match_type == "혼성복식" and match_type_count[player]["혼성복식"] >= 2:
                return False
            if match_type == "남자복식":
                if match_type_count[player]["남자복식"] == 3 and match_type_count[player]["혼성복식"] == 1:
                    return False
                if match_type_count[player]["남자복식"] == 2 and match_type_count[player]["혼성복식"] == 2:
                    return False
    return True

# 가능한 매치 조합을 미리 생성
mixed_double_matches = [
    list(combo) for men_combo in combinations(men, 2) for women_combo in combinations(women, 2)
    for combo in [men_combo + women_combo]
]
men_double_matches = [list(combo) for combo in combinations(men, 4)]
women_double_matches = [list(combo) for combo in combinations(women, 4)]

# 경기 스케줄 생성
def create_schedule():
    schedule = []

    for _ in range(total_rounds):
        round_matches = []
        
        # 각 코트에 맞는 매치 타입을 미리 배정
        court_matches = random.choices(["혼성복식", "남자복식", "여자복식"], k=num_courts)

        for match_type in court_matches:
            # 각 매치 타입에 맞는 가능한 매치 리스트에서 조건을 충족하는 매치를 선택
            allow_flexible_pairs = False
            if match_type == "혼성복식":
                valid_matches = [match for match in mixed_double_matches if is_valid_match(match, "혼성복식", allow_flexible_pairs)]
                if not valid_matches:
                    allow_flexible_pairs = True  # 조건 완화
                    valid_matches = [match for match in mixed_double_matches if is_valid_match(match, "혼성복식", allow_flexible_pairs)]
                if valid_matches:
                    match = random.choice(valid_matches)
                    mixed_double_matches.remove(match)
            elif match_type == "남자복식":
                valid_matches = [match for match in men_double_matches if is_valid_match(match, "남자복식", allow_flexible_pairs)]
                if not valid_matches:
                    allow_flexible_pairs = True
                    valid_matches = [match for match in men_double_matches if is_valid_match(match, "남자복식", allow_flexible_pairs)]
                if valid_matches:
                    match = random.choice(valid_matches)
                    men_double_matches.remove(match)
            elif match_type == "여자복식":
                valid_matches = [match for match in women_double_matches if is_valid_match(match, "여자복식", allow_flexible_pairs)]
                if not valid_matches:
                    allow_flexible_pairs = True
                    valid_matches = [match for match in women_double_matches if is_valid_match(match, "여자복식", allow_flexible_pairs)]
                if valid_matches:
                    match = random.choice(valid_matches)
                    women_double_matches.remove(match)
            else:
                continue

            # 매치 기록 및 경기 배정
            record_match(match, match_type)
            round_matches.append((match_type, match))

        schedule.append(round_matches)

    # 각 선수의 경기 횟수가 정확히 4번인지 확인
    for player in players_matches:
        if players_matches[player] != total_matches_per_player:
            print(f"경고: {player}가 {players_matches[player]}번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)")

    return schedule

# 스케줄 생성 및 출력
schedule = create_schedule()

if schedule:
    for round_num, round_matches in enumerate(schedule, start=1):
        print(f"Round {round_num}")
        for court_num, (match_type, players) in enumerate(round_matches, start=1):
            print(f"  Court {court_num}: {match_type} - {players}")

    # 각 선수별 경기 횟수 및 경기 타입 확인
    for player, count in players_matches.items():
        print(f"{player} has {count} matches (혼성복식: {match_type_count[player]['혼성복식']}, 남자복식: {match_type_count[player]['남자복식']}, 여자복식: {match_type_count[player]['여자복식']})")
else:
    print("스케줄 생성에 실패했습니다. 조건을 충족하는 구성을 찾을 수 없습니다.")


경고: 남3가 5번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 남4가 5번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 남10가 5번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 남12가 2번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 남16가 5번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 남17가 2번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 남18가 5번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 남21가 5번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 여1가 3번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 여4가 5번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 여5가 3번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 여6가 2번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
경고: 여9가 5번의 경기에 배정되었습니다. (4경기를 초과 또는 미달)
Round 1
  Court 1: 혼성복식 - ['남2', '남6', '여2', '여10']
  Court 2: 여자복식 - ['여3', '여4', '여5', '여8']
  Court 3: 여자복식 - ['여2', '여8', '여9', '여10']
  Court 4: 남자복식 - ['남2', '남4', '남11', '남15']
Round 2
  Court 1: 혼성복식 - ['남1', '남9', '여5', '여6']
  Court 2: 여자복식 - ['여3', '여4', '여7', '여9']
  Court 3: 여자복식 - ['여1', '여2', '여7', '여10']
  Court 4: 남자복식 - ['남2', '남6', '남18', '남19']
Round 3
  Court 1: 여자복식 - ['남2', '남6', '남18', '남19']
  Court 2: 혼성복식 - ['남11', '남16', '여5', '여

In [30]:
import random
from itertools import combinations

# 선수 목록 생성
men = [f"남{i+1}" for i in range(22)]
women = [f"여{i+1}" for i in range(10)]

# 설정
total_matches_per_player = 4
num_courts = 4
total_rounds = 8

# 선수 별 경기 유형 및 목표 경기 수 설정
player_targets = {player: {'혼성복식': 2 if player in women else 1, 
                           '남자복식': 3 if player in men else 0, 
                           '여자복식': 2 if player in women else 0}
                  for player in men + women}
matches_assigned = {player: 0 for player in men + women}
partner_counts = {player: {} for player in men + women}

# 파트너 만남 횟수를 기록하는 함수
def record_match(players, match_type):
    for player in players:
        matches_assigned[player] += 1
        player_targets[player][match_type] -= 1
        for partner in players:
            if partner != player:
                if partner not in partner_counts[player]:
                    partner_counts[player][partner] = 0
                partner_counts[player][partner] += 1

# 조건 충족 확인 함수
def is_valid_match(players, match_type):
    for player in players:
        # 남은 배정할 경기 수가 있는지 확인
        if player_targets[player][match_type] <= 0:
            return False
        # 각 선수의 총 경기 횟수가 4회 이내인지 확인
        if matches_assigned[player] >= total_matches_per_player:
            return False
        # 파트너와의 만남 횟수를 3회로 제한
        if any(partner_counts[p1].get(p2, 0) >= 3 for p1 in players for p2 in players if p1 != p2):
            return False
    return True

# 가능한 매치 조합을 미리 생성
mixed_double_matches = [
    list(combo) for men_combo in combinations(men, 2) for women_combo in combinations(women, 2)
    for combo in [men_combo + women_combo]
]
men_double_matches = [list(combo) for combo in combinations(men, 4)]
women_double_matches = [list(combo) for combo in combinations(women, 4)]

# 경기 스케줄 생성
def create_schedule():
    schedule = []

    for _ in range(total_rounds):
        round_matches = []
        
        # 각 코트에 맞는 매치 타입을 미리 배정
        court_matches = random.choices(["혼성복식", "남자복식", "여자복식"], k=num_courts)

        for match_type in court_matches:
            # 각 매치 타입에 맞는 가능한 매치 리스트에서 조건을 충족하는 매치를 선택
            if match_type == "혼성복식":
                valid_matches = [match for match in mixed_double_matches if is_valid_match(match, "혼성복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    mixed_double_matches.remove(match)
            elif match_type == "남자복식":
                valid_matches = [match for match in men_double_matches if is_valid_match(match, "남자복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    men_double_matches.remove(match)
            elif match_type == "여자복식":
                valid_matches = [match for match in women_double_matches if is_valid_match(match, "여자복식")]
                if valid_matches:
                    match = random.choice(valid_matches)
                    women_double_matches.remove(match)
            else:
                continue

            # 매치 기록 및 경기 배정
            record_match(match, match_type)
            round_matches.append((match_type, match))

        schedule.append(round_matches)

    # 각 선수의 경기 횟수가 정확히 4번인지 확인
    for player in matches_assigned:
        if matches_assigned[player] != total_matches_per_player:
            print(f"경고: {player}가 {matches_assigned[player]}번의 경기에 배정되었습니다.")

    return schedule

# 스케줄 생성 및 출력
schedule = create_schedule()

if schedule:
    for round_num, round_matches in enumerate(schedule, start=1):
        print(f"Round {round_num}")
        for court_num, (match_type, players) in enumerate(round_matches, start=1):
            print(f"  Court {court_num}: {match_type} - {players}")

    # 각 선수별 경기 횟수 및 경기 타입 확인
    for player, count in matches_assigned.items():
        print(f"{player} has {count} matches (혼성복식: {2 - player_targets[player]['혼성복식']}, 남자복식: {3 - player_targets[player]['남자복식'] if player in men else 0}, 여자복식: {2 - player_targets[player]['여자복식'] if player in women else 0})")
else:
    print("스케줄 생성에 실패했습니다. 조건을 충족하는 구성을 찾을 수 없습니다.")


경고: 남1가 3번의 경기에 배정되었습니다.
경고: 남6가 5번의 경기에 배정되었습니다.
경고: 남7가 5번의 경기에 배정되었습니다.
경고: 남11가 5번의 경기에 배정되었습니다.
경고: 남12가 7번의 경기에 배정되었습니다.
경고: 남14가 6번의 경기에 배정되었습니다.
경고: 남16가 3번의 경기에 배정되었습니다.
경고: 남18가 2번의 경기에 배정되었습니다.
경고: 남21가 6번의 경기에 배정되었습니다.
경고: 여1가 2번의 경기에 배정되었습니다.
경고: 여2가 2번의 경기에 배정되었습니다.
경고: 여7가 3번의 경기에 배정되었습니다.
경고: 여8가 3번의 경기에 배정되었습니다.
Round 1
  Court 1: 남자복식 - ['남4', '남7', '남11', '남21']
  Court 2: 남자복식 - ['남6', '남17', '남20', '남22']
  Court 3: 혼성복식 - ['남4', '남13', '여4', '여6']
  Court 4: 여자복식 - ['여5', '여8', '여9', '여10']
Round 2
  Court 1: 여자복식 - ['여3', '여4', '여8', '여9']
  Court 2: 여자복식 - ['여3', '여5', '여6', '여7']
  Court 3: 혼성복식 - ['남2', '남18', '여1', '여4']
  Court 4: 남자복식 - ['남1', '남4', '남8', '남18']
Round 3
  Court 1: 남자복식 - ['남12', '남13', '남16', '남22']
  Court 2: 남자복식 - ['남3', '남10', '남13', '남17']
  Court 3: 남자복식 - ['남2', '남10', '남13', '남16']
  Court 4: 남자복식 - ['남4', '남9', '남10', '남11']
Round 4
  Court 1: 남자복식 - ['남8', '남11', '남17', '남20']
  Court 2: 여자복식 - ['여1', '여4', '여6', '여10']
  Court 3: